In [1]:
%load_ext autoreload
%autoreload 2

import sys

In [2]:
import os
import array
import re

import pandas
import numpy

import plotly.express
import librosa
from matplotlib import pyplot as plt

from software.features.spectral import design_bandpass, plot_bandpass
from software.features.featureutils import resample, rms, vector_magnitude
from software.dataset.combine import load_data, read_labels, parse_video_filename, apply_labels

In [3]:
combined_path = 'data/jonnor-brushing-1/combined.parquet'

samplerate = 50
lower = 2.0
upper = 5.0

In [4]:
data = pandas.read_parquet(combined_path)
data

,level_0,x,y,z,filename,dummy_filename,class,session,is_brushing
time,,,,,,,,,
2024-12-31 15:45:19.000,0,-0.480972,0.283700,0.852565,2024-12-31T154519_brushing.npy,only-one-sensor,None,VID_20241231_154514.mkv,False
2024-12-31 15:45:19.020,0,-0.483413,0.277841,0.853053,2024-12-31T154519_brushing.npy,only-one-sensor,None,VID_20241231_154514.mkv,False
2024-12-31 15:45:19.040,0,-0.479995,0.271981,0.842311,2024-12-31T154519_brushing.npy,only-one-sensor,None,VID_20241231_154514.mkv,False
2024-12-31 15:45:19.060,0,-0.484390,0.275399,0.868191,2024-12-31T154519_brushing.npy,only-one-sensor,None,VID_20241231_154514.mkv,False
2024-12-31 15:45:19.080,0,-0.487808,0.274911,0.846706,2024-12-31T154519_brushing.npy,only-one-sensor,None,VID_20241231_154514.mkv,False
...,...,...,...,...,...,...,...,...,...
2024-12-31 16:49:52.900,0,0.092288,0.981475,-0.120121,2024-12-31T164948_brushing.npy,only-one-sensor,None,VID_20241231_164646.mkv,False
2024-12-31 16:49:52.920,0,-0.097171,1.007355,-0.035646,2024-12-31T164948_brushing.npy,only-one-sensor,None,VID_20241231_164646.mkv,False
2024-12-31 16:49:52.940,0,0.159185,0.978545,-0.065920,2024-12-31T164948_brushing.npy,only-one-sensor,None,VID_20241231_164646.mkv,False


In [ ]:
def load_classifier(path):
    
    

In [15]:
data_columns = ['x', 'y', 'z']
sensitivity = 2.0
for session, session_data in data.groupby('session'):
    print(session)
    d = session_data.sort_index().reset_index()[data_columns]
    p =  f'{session}.npy'
    scaled = ((d.values / sensitivity) * (2**15-1)).astype(numpy.int16)
    arr = numpy.ascontiguousarray(scaled)
    #print(arr)
    numpy.save(p, arr, allow_pickle=True)
    print(p)
    # TODO: run firmware/process.py and get the output


VID_20241231_154514.mkv
VID_20241231_154514.mkv.npy
VID_20241231_155624.mkv
VID_20241231_155624.mkv.npy
VID_20241231_163515.mkv
VID_20241231_163515.mkv.npy
VID_20241231_164646.mkv
VID_20241231_164646.mkv.npy


In [ ]:

def plot_session(sensors, labels, title=None, height=300, aspect=3.0):

    time_column = 'time'
    columns = ['x', 'y', 'z']
    width = height * aspect
    annotation_column = 'class'
    
    # TODO: plot other values than raw XYZ.
    # Like motion, from removing gravity. Maybe overall RMS, delta rms, band-passed 2-5hz instead of  
    sel = sensors.reset_index()
    fig = plotly.express.line(sel,
                              x=time_column,
                              y=columns,
                              width=width,
                              height=height,
                              title=title,
                             )
    fig.update_traces(connectgaps=False)
    fig.update_layout(showlegend=False)

    colors = plotly.express.colors.qualitative.Plotly

    types = labels[annotation_column].unique()
    annotation_colors = { v: c for v, c in zip(types, colors) }
    
    # FIXME: put this on the bottom
    for idx, l in labels.iterrows():
        #print(dict(l))
        color = annotation_colors[l[annotation_column]]
        fig.add_vrect(x0=l['start_time'],
                x1=l['end_time'],
                line_width=0,
                fillcolor=color,
                opacity=0.3,
                label=dict(
                    text=l[annotation_column],
                    textposition="start",
                    font=dict(size=10, color="black"),
                    yanchor="top",
                ),
         )
    
    return fig

for session, session_data in ss.groupby('session'):
    print(session)
    ll = mm.set_index('filename').loc[session]
    fig = plot_session(session_data, ll, title=session)
    fig.show()

In [ ]:
print(m.elapsed.max())